In [22]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as sc_stats
import scipy.optimize as sc_optimize
import tabulate
import lmfit

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

import plotly.graph_objects as go

from odf.opendocument import load as odf_load

In [16]:
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({
  'font.size': 16,
  'grid.alpha': 0.25})


## Function declarations

### Read `ods` to `DataFrame`

In [3]:
def read_ods(filename, columns=None):
  if columns is None:
    df = pd.read_excel(filename, engine="odf", header=0)
  elif isinstance(columns, list):
    df = pd.read_excel(filename, engine="odf", header=0,
                       usecols=columns)
  else:
    key_arr = (k for k in columns.keys())
    df = pd.read_excel(filename, engine="odf", header=0,
                      usecols=key_arr)
    df = df.rename(columns=columns)
  return df

### Get multiple regression coefficients

In [4]:
def get_multiple_regress_coeff(df, n_vars=2, with_intercept=True):  

  regress_obj = LinearRegression(fit_intercept=with_intercept)

  if isinstance(df, pd.DataFrame):
    num_of_samples = len(df)
    X_arr = np.zeros((num_of_samples, n_vars))
    Y = df["Y"].to_numpy()

    for i in range(n_vars):
      key = f"X_{i+1}"
      X_arr[:, i] = df[key].to_numpy()

    regress = regress_obj.fit(X_arr, Y)
    df["hat_Y"] = regress.intercept_ + X_arr.dot(regress.coef_) 
  
  elif isinstance(df, np.ndarray):
    Y = df[:, 0]
    X_arr = df[:, 1:]

    
    regress = regress_obj.fit(X_arr, Y)
    Y_hat = regress.intercept_ + X_arr.dot(regress.coef_)
    df = np.column_stack([df, Y_hat])

  return regress, df


### Get Minitab output of Regression Analysis

In [5]:
def get_minitab_out(df, n_vars=2, new_column=None, withCorrMatrix=False, 
  withVIF=False, with_intercept=True, is_reg_diff=False):

  """
  new_column (dict) : a dictionary mapping to rename df header
  is_reg_diff:
    True - if we perform regression difference of Y_t and X_t
  """
  
  data_struct = np.zeros((len(df), 1+n_vars))
  data_struct[:, 0] = df['Y']
  for i in range(n_vars):
    key = f"X_{i+1}"
    data_struct[:, i+1] = df[key]

  # -- compute correlation matrix
  corr_matrix = np.corrcoef(data_struct, rowvar=False)
  corr_matrix = corr_matrix[1:, 0:-1]    # only show lower triangle part of correlation matrix

  # -- compute regression intercept and coefficients
  regress, _ = get_multiple_regress_coeff(df, n_vars=n_vars, with_intercept=with_intercept)
  b_arr = [regress.intercept_] + regress.coef_.tolist()
  is_positive_b_arr = [b_j > 0 for b_j in b_arr]
  #print(f"b_arr: {b_arr}")
  
  # -- compute standad error of the estimates
  num_of_samples = len(df)
  hat_Y =  regress.intercept_ + data_struct[:,1:].dot(regress.coef_)
  sumSq_Y_hat_Y = ((data_struct[:, 0] - hat_Y)**2).sum()

  if with_intercept:
    s_yxs = np.sqrt(sumSq_Y_hat_Y/(num_of_samples - n_vars - 1))
  else:
    s_yxs = np.sqrt(sumSq_Y_hat_Y/(num_of_samples - n_vars))

  # -- compute cofactor matrix 
  if with_intercept:
    X_arr = np.ones_like(data_struct)
    X_arr[:,1:] = data_struct[:,1:]
    cofactor_matrix = np.linalg.inv(X_arr.transpose().dot(X_arr))   # this matrix is closely related to covariance matrix
  else:
    X_arr = data_struct[:, 1:]
    cofactor_matrix = np.linalg.inv(X_arr.transpose().dot(X_arr))   # this matrix is closely related to covariance matrix
  

  # -- compute standard error of intercept_ and coef_
  # -- compute t-score of intecept_ and coef_
  # -- compute p-values of intercept_ and coef_
  SE_coef = np.zeros(1+n_vars)
  t_scores = np.zeros(1+n_vars)
  p_values = np.zeros(1+n_vars)
  
  if with_intercept:
    dof = num_of_samples - n_vars - 1;          # degrees of freedom
    for i in range(1+n_vars):
      SE_coef[i] = s_yxs*np.sqrt(cofactor_matrix[i, i])
      t_scores[i] = b_arr[i] / SE_coef[i]
      p_values[i] = sc_stats.t.sf(abs(t_scores[i]), dof) * 2   # .t.sf is a survival function (1 - cdf)
  else:
    dof = num_of_samples - n_vars;          # degrees of freedom
    for i in range(1, 1+n_vars):
      SE_coef[i] = s_yxs*np.sqrt(cofactor_matrix[i-1, i-1])
      t_scores[i] = b_arr[i] / SE_coef[i]
      p_values[i] = sc_stats.t.sf(abs(t_scores[i]), dof) * 2   # .t.sf is a survival function (1 - cdf)

  # -- compute VIF of coef_
  if withVIF:
    # if n_vars == 2, we calculate VIF from correlation matrix
    VIF_arr = np.zeros(n_vars)
    if n_vars == 2:
      VIF_arr[0] = 1/(1-corr_matrix[1, 1]**2)
      VIF_arr[1] = 1/(1-corr_matrix[1, 1]**2)
    else:
      for i in range(n_vars):
        # take a predictor column and put the other predictor after it
        predictor_data_struct = np.zeros((num_of_samples, n_vars))
        predictor_data_struct[:,0] = data_struct[:, i+1]   
        predictor_data_struct[:,1:] = np.delete(data_struct, i+1, axis=1)[:,1:]  
        #print(predictor_data_struct)
        predictor_SST, predictor_SSR, _ = get_sumSq(predictor_data_struct, n_vars=n_vars)
        predictor_R_sq = predictor_SSR/predictor_SST
        #print(predictor_R_sq)
        VIF_arr[i] = 1/(1 - predictor_R_sq)


  # -- compute ANOVA table
  meanY = data_struct[:, 0].mean()
  if with_intercept:
    SSR = ((hat_Y - meanY)**2).sum()
    SST = ((data_struct[:, 0] - meanY)**2).sum()
  else:
    SSR = (hat_Y**2).sum()
    SST = (data_struct[:, 0]**2).sum()

  SSE = ((data_struct[:, 0] - hat_Y)**2).sum()
  anova_sumSq = np.array([SSR, SSE, SST])

  if with_intercept:
    anova_dof = np.array([n_vars, num_of_samples - n_vars - 1, num_of_samples - 1], dtype=int)
  else:
    anova_dof = np.array([n_vars, num_of_samples - n_vars, num_of_samples], dtype=int)
  
  anova_meanSq = (anova_sumSq/anova_dof)[:2]
  anova_F_score = anova_meanSq[0] / anova_meanSq[1]
  anova_p_value = sc_stats.f.sf(anova_F_score, anova_dof[0], anova_dof[1])   # sf is a surfifal function (1 - cdf)

  # -- compute R_sq (coefficient of determination)
  R_sq = SSR/SST

  # -- compute adjusted R_sq
  adj_R_sq = 1 - (1 - R_sq)*(num_of_samples-1)/(num_of_samples-n_vars-1)
  
  # -- calculate Durbin-Watson statistics
  e_t = data_struct[:, 0] - hat_Y
  numerator = ((e_t[1:] - e_t[:-1])**2).sum()
  dw_stats = numerator / (e_t**2).sum()

  # -- create tabular form for correlation 
  if withCorrMatrix:
    data = []
    for i in range(n_vars):
      data_row = [new_column[i+1]] + \
        [f"{corr:.3f}" if j < i+1 else "" for j, corr in enumerate(corr_matrix[i,:])]
      #print(data_row)
      #print([type(data_row_i) for data_row_i in data_row])
      data.append(data_row)

    table_corr = tabulate.tabulate(data, tablefmt='html', 
      headers=[""] + new_column[:n_vars], 
      floatfmt=["None"] + [".3f"]*(n_vars))

    print(f"Correlations: {', '.join(new_column)}")
    display(table_corr)

  # -- create tabular form for predictor
  data = [["Constant", b_arr[0], SE_coef[0], t_scores[0], p_values[0]]]
  if withVIF:
    for i in range(1, n_vars+1):
      data_row = [new_column[i], b_arr[i], SE_coef[i], t_scores[i], p_values[i], VIF_arr[i-1]]
      data.append(data_row)
    table_predictor = tabulate.tabulate(data, tablefmt='html', 
      headers=["Predictor", "Coef", "SE Coef", "t-score", "p-value", "VIF"], 
      floatfmt=[None, ".4f", ".4f", ".2f", ".4f", ".3f"])
  else:
    for i in range(1, n_vars+1):
      data_row = [new_column[i], b_arr[i], SE_coef[i], t_scores[i], p_values[i]]
      data.append(data_row)
    table_predictor = tabulate.tabulate(data, tablefmt='html', 
      headers=["Predictor", "Coef", "SE Coef", "t-score", "p-value"], 
      floatfmt=[None, ".4f", ".4f", ".2f", ".4f"])


  print(f"Regression Analysis: {new_column[0]} versus {', '.join(new_column[1:])}")
  print("The regression equation is")
  str_b_predictor = "" 
  for i in range(n_vars):
    sign = "+" if is_positive_b_arr[i+1] else "-"
    str_b_predictor += f" {sign} {abs(b_arr[i+1]):.3f} {new_column[i+1]}"
  if with_intercept:
    print(f"{new_column[0]} = {b_arr[0]:.3f}{str_b_predictor}")
  else:
    if str_b_predictor[1] == "+":
      str_b_predictor = str_b_predictor[3:]
    print(f"{new_column[0]} = {str_b_predictor}")
  display(table_predictor)
  if is_reg_diff:
    print(f"s_yxs = {s_yxs:.4f}")
  else:
    print(f"s_yxs = {s_yxs:.4f}   R_sq = {R_sq*100:.1f}%   R-sq(adj) = {adj_R_sq*100:.1f}%")

  # -- create ANOVA table (with F-score and its p-value)
  data = [
    ["Regression",     anova_dof[0], anova_sumSq[0], f"{anova_meanSq[0]:.4f}", f"{anova_F_score:.3f}", f"{anova_p_value:.4f}"],
    ["Residual error", anova_dof[1], anova_sumSq[1], f"{anova_meanSq[1]:.4f}", "", ""],
    ["Total",          anova_dof[2], anova_sumSq[2], "", "", ""]] 
  table_anova = tabulate.tabulate(data, tablefmt='html', 
    headers=["Source", "d.o.f", "sumSq", "meanSq", "F-score", "p-value"], 
    floatfmt=[None, ".0f", ".4f", "s", "s", "s"])

  print(f"Analyis of Variance")
  display(table_anova)

  # -- print Durbin-Watson statistics
  print(f"Durbin-Watson Statistics = {dw_stats:.2f}")

  return {"regress": regress, "corr_matrix": corr_matrix, 
    "R_sq": R_sq, "dw_stats": dw_stats}


### Perform Durbin-Watson Test

In [6]:
def test_statistic_dw(dw_stats, n_samples, k_predictors, significance_lvl_arr):
  for significance_lvl in significance_lvl_arr:
    if abs(significance_lvl - 0.05) < 1e-4:
      d_lower = df_dw_crit_05.xs(n_samples).xs(f"k = {k_predictors}").xs("d_L")
      d_upper = df_dw_crit_05.xs(n_samples).xs(f"k = {k_predictors}").xs("d_U")
      
    elif abs(significance_lvl - 0.01) < 1e-4:
      d_lower = df_dw_crit_01.xs(n_samples).xs(f"k = {k_predictors}").xs("d_L")
      d_upper = df_dw_crit_01.xs(n_samples).xs(f"k = {k_predictors}").xs("d_U")

    test_status = "" 
    if dw_stats > d_upper:
      test_status = "null hypothesis accepted"
    elif dw_stats < d_lower:
      test_status = "null hypotesis rejected"
    else:
      test_status = "inconclusive"

    print(f"significance_lvl: {significance_lvl}")
    print(f"  durbin-watson statistics: {dw_stats:.2f}")
    print(f"  durbin-watson critical values (d_L, d_U): {d_lower, d_upper}")
    print(f"  conclusion: {test_status}\n")

### Durbin-Watson critical value

In [7]:
df_dw_crit_05 = pd.read_excel("../durbin-watson-test-bounds-alpha-0_05.ods",
  header=[1, 2], index_col=0)
df_dw_crit_01 = pd.read_excel("../durbin-watson-test-bounds-alpha-0_01.ods", 
  header=[1, 2], index_col=0)

## Load data

### Sales and Income data

In [8]:
filename = "07-table-05-sears-sales.ods"

df = read_ods(filename)

df["ln Y_t"] = np.log(df["Sales ($1,000s) Y_t"])
df["ln X_t"] = np.log(df["Income ($ millions) X_t"])

df["diff ln Y_t"] = df["ln Y_t"] - df["ln Y_t"].shift(1)
df["diff ln X_t"] = df["ln X_t"] - df["ln X_t"].shift(1)

df

,Year,"Sales ($1,000s) Y_t",Income ($ millions) X_t,ln Y_t,ln X_t,diff ln Y_t,diff ln X_t
0,1976,3307,273.4,8.103797,5.610936,NaN,NaN
1,1977,3556,291.3,8.176392,5.674354,0.072595,0.063418
2,1978,3601,306.9,8.188967,5.726522,0.012575,0.052168
3,1979,3721,317.1,8.221748,5.759217,0.032781,0.032695
4,1980,4036,336.1,8.303009,5.817409,0.081262,0.058192
5,1981,4134,349.4,8.327001,5.856217,0.023991,0.038809
6,1982,4268,362.9,8.358901,5.894127,0.031900,0.037910
7,1983,4578,383.9,8.429018,5.950382,0.070117,0.056255
8,1984,5093,402.8,8.535622,5.998440,0.106605,0.048058
9,1985,5716,437.0,8.651025,6.079933,0.115402,0.081493


## Simple Linear Regression

### Regression Analysis

In [9]:
df_minitab = df[["Sales ($1,000s) Y_t", "Income ($ millions) X_t"]]
from_column_names = df_minitab.columns.to_list()
print(f"header: {from_column_names}")
to_column_names = ["Y"] + [f"X_{i+1}" for i in range(len(from_column_names) - 1)]


df_minitab = df_minitab.rename(columns=
  {k: v for k, v in zip(from_column_names, to_column_names)})

df_minitab


header: ['Sales ($1,000s) Y_t', 'Income ($ millions) X_t']


,Y,X_1
0,3307,273.4
1,3556,291.3
2,3601,306.9
3,3721,317.1
4,4036,336.1
5,4134,349.4
6,4268,362.9
7,4578,383.9
8,5093,402.8
9,5716,437.0


In [10]:
column_name = ["Sales", "Income"]
out_dict = get_minitab_out(df_minitab, n_vars=len(column_name)-1, 
  new_column=column_name, withVIF=False, withCorrMatrix=False)


Regression Analysis: Sales versus Income
The regression equation is
Sales = -524.332 + 14.050 Income


Predictor,Coef,SE Coef,t-score,p-value
Constant,-524.3323,188.4470,-2.78,0.0119
Income,14.0496,0.3185,44.11,0.0000


s_yxs = 343.5060   R_sq = 99.0%   R-sq(adj) = 99.0%
Analyis of Variance


Source,d.o.f,sumSq,meanSq,F-score,p-value
Regression,1,229603716.2712,229603716.2712,1945.854,0.0000
Residual error,19,2241930.9669,117996.3667,,
Total,20,231845647.2381,,,


Durbin-Watson Statistics = 0.63


### Durbin-Watson Test

The *Durbin-Watson test* can be used to determine whether positive lag 1
autocorrelation is present.

$$
  \begin{gather*}
    \text{If } DW > d_\mathrm{U}, \text{ conclude } H_0: \rho = 0. \\
    \text{If } DW < d_\mathrm{L}, \text{ conclude } H_1: \rho > 0.
  \end{gather*}
$$

If $DW$ lies withing the lower and upper bounds $(d_\mathrm{L} \leq DW \leq d_\mathrm{U})$,
the test is inconclusive.

In [11]:
significance_lvl = [0.05, 0.01]
n_samples = len(df_minitab) 
k_predictors = 1
dw_stats = out_dict["dw_stats"]

test_statistic_dw(dw_stats, n_samples, k_predictors, significance_lvl)

significance_lvl: 0.05
  durbin-watson statistics: 0.63
  durbin-watson critical values (d_L, d_U): (1.22, 1.42)
  conclusion: null hypotesis rejected

significance_lvl: 0.01
  durbin-watson statistics: 0.63
  durbin-watson critical values (d_L, d_U): (0.97, 1.16)
  conclusion: null hypotesis rejected



With a significant level $5\%$, the DW statistic is close to zero (null hypotesis is rejected). The positive autocorrelation is indicated.

## Generalized difference model

$$
\begin{align*}
  Y'_t = \beta_0 (1 - \rho) + \beta_1 X'_t + \nu_t
\end{align*}
$$

with $Y'_t = Y_t - \rho Y_{t-1}$ and $X'_t = X_t - \rho X_{t-1}$

In [36]:
df_generalized = df_minitab[["Y", "X_1"]]

df_generalized["diff Y"] = df_generalized["Y"] - df_generalized["Y"].shift(1)

df_generalized

,Y,X_1,diff Y
0,3307,273.4,NaN
1,3556,291.3,249.0
2,3601,306.9,45.0
3,3721,317.1,120.0
4,4036,336.1,315.0
5,4134,349.4,98.0
6,4268,362.9,134.0
7,4578,383.9,310.0
8,5093,402.8,515.0
9,5716,437.0,623.0


In [77]:
a = np.array([1, 2, 3])
a = np.hstack((a, 2))
a

array([1, 2, 3, 2])

In [131]:
def residual_func(params, x, y_data, x0, y0_data):
  """The following definition of residual_func is a little bit trickier.
     We have to separate the first value of x and y_data to make input/output
     array of x and y_data have the same dimension"""
  beta0 = params["beta0"].value
  beta1 = params["beta1"].value
  rho = params["rho"].value

  diff_x = np.zeros_like(x, dtype=float)
  diff_x[1:] = x[1:] - rho*x[:-1]
  diff_x[0] = x[0] - rho*x0
  
  diff_y_data = np.zeros_like(y_data, dtype=float)
  diff_y_data[1:] = y_data[1:] - rho*y_data[:-1]
  diff_y_data[0] = y_data[0] - rho*y0_data
  
  return diff_y_data - beta0*(1. - rho) - beta1*diff_x

def least_squares(df):
  params = lmfit.Parameters()
  params.add("beta0", 50_000)
  params.add("beta1", 5)
  params.add("rho", 0.5)

  x = df["X_1"].to_numpy()
  x0 = x[0]
  x = x[1:]
  y_data = df["Y"].to_numpy()
  y0_data = y_data[0]
  y_data = y_data[1:]
  ol = lmfit.minimize(residual_func, params, args=(x, y_data, x0, y0_data), 
    method="leastsq", ftol=1e-20)

  df["hat_Y"] = df["Y"] + np.hstack((np.nan, ol.residual))

  return ol, df

In [132]:
regress_generalized, df_regress  = least_squares(df_generalized)
regress_generalized

In [99]:
df_regress

,Y,X_1,diff Y,hat_Y
0,3307,273.4,NaN,NaN
1,3556,291.3,249.0,3491.217853
2,3601,306.9,45.0,3353.753623
3,3721,317.1,120.0,3598.498474
4,4036,336.1,315.0,4027.053960
5,4134,349.4,98.0,3961.238998
6,4268,362.9,134.0,4129.316513
7,4578,383.9,310.0,4545.868090
8,5093,402.8,515.0,5285.640751
9,5716,437.0,623.0,5875.857810


In [17]:
def get_eviews_out(df, n_vars=3):
  return None